<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021ccap_word_association_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021ccap word2vec による単語連想課題のデモ

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 05/Feb/2021<br>
 MIT license
</div>

In [ ]:
# -*- coding: utf-8 -*-
from six.moves.urllib import request

import numpy as np
import os
import sys
import time
import gensim

In [ ]:
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz'
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.g

In [ ]:
word2vec_file='2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file, 
                                                        encoding='utf-8', 
                                                        unicode_errors='replace', 
                                                        binary=True)
w2v = model

In [ ]:
# まずは word2vec の確認
w2v.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)

In [ ]:
# 読み込んだモデルは日本語モデルなので日本語で検討
w2v.most_similar(positive=['王','女'], negative=['男'], topn=10)

In [ ]:
# 今回の元になった論文による結果
from IPython.display import SVG, display, HTML
github_base = 'https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/'
SVG(url=github_base+'2018Rotaru_fig3.svg')

In [ ]:
!pip install --upgrade jaconv japanize_matplotlib 

In [ ]:
import time

In [ ]:
def free_word_assoc(word, topn=10, beta=0.03, n_seq=10, seed=int(time.time()), verbose=False, w2v=w2v):
    """
    自由連想課題のシミュレーション: 単語 word を刺激語として逐次自由連想を繰り返す
    パラメータ:
        word: 刺激語 
        topn: 想起する単語の最大数，上位 topn 個の単語までが考慮される(int)
        beta: 温度パラメータ，低いと決定論的に振る舞い，高いとその都度振る舞いが異なる(float)
        seed: 乱数の種 指定しなければ現在の時刻を用いる。指定すると再現性が保たれる (int) 
        verbose: 冗長な出力を行う (bool)
        w2v: word2vec の本体
    """
    np.random.seed(seed)
    ret = []
    for i in range(n_seq):
        vec = w2v.similar_by_word(word, topn=topn)
        sims = np.array([x[1] for x in vec][1:])
        names = [x[0] for x in vec][1:]
        prob = np.exp(sims/beta) / np.exp(sims/beta).sum()
        word = np.random.choice(names,p=prob)
        ret.append(word)
        if verbose:
            print(word, end="")
            print(' :', names.index(word)+1, names, prob)
    return ret

free_word_assoc('ネコ', n_seq=5, beta=0.02, verbose=False)

In [ ]:
free_word_assoc('NTT')

In [ ]:
free_word_assoc('東京')


In [ ]:
free_word_assoc('女性')

In [ ]:
for  word in 'computer', 'コンピュータ':
    print(free_word_assoc(word))

In [ ]:
for word in 'コーヒー', '紅茶':
    print(free_word_assoc(word,n_seq=3))

In [ ]:
for word in 'ビール', '日本酒':
    print(free_word_assoc(word,n_seq=5))